# Introduction

This project will focus on exploring the capabilities of Bayesian optimization, specifically employing BayBE, in the discovery of novel corrosion inhibitors for materials design. Initially, we will work with a randomly chosen subset from a comprehensive database of electrochemical responses of small organic molecules. Our goal is to assess how Bayesian optimization can speed up the screening process across the design space to identify promising compounds. We will compare different strategies for incorporating alloy information, while optimizing the experimental parameters with respect to the inhibitive performance of the screened compounds.

# Initizalization

Loading libraries and data files:

In [ ]:
import pandas as pd
import numpy as np

df_AA2024 = pd.read_excel('data/filtered_AA2024.xlsx')
# df_AA1000 = pd.read_excel('data/filtered_AA1000.xlsx')
# df_Al = pd.read_excel('data/filtered_Al.xlsx')

In [ ]:
print(df_AA2024.describe())

In [ ]:
print(df_AA2024.head())

# Data Processing

### Extract all unique SMILES values into dictionary

In [ ]:
unique_SMILES = df_AA2024.SMILES.unique()

def list_to_dict(input_list):
    return {item: item for item in input_list}

smiles_dict =list_to_dict(unique_SMILES)

# Data Anaylsis

# Bayesian Optimization

## Search Space

In [ ]:
from baybe.parameters import NumericalDiscreteParameter, NumericalContinuousParameter
from baybe.searchspace import SearchSpace

parameters = [
NumericalDiscreteParameter(
    name="Time (h)",
    values=np.arange(1, 25, 1)
    # tolerance = 0.004, assume certain experimental noise for each parameter measurement?
),
NumericalDiscreteParameter(
    name="pH",
    values=np.arange(-1, 15.1, 0.1)
    # tolerance = 0.004
    ),  
NumericalDiscreteParameter( # Set this as continuous, the values seem quite small?
    name="Inhibitor Concentration (M)",
    values=np.arange(0, 0.1, 0.01), # Remove data outliers like 0.1?
    # tolerance = 0.004
    ),
NumericalDiscreteParameter(
    name="Salt Concentration (M)",
    values=np.arange(0, 2.01, 0.01),
    # tolerance = 0.004
    )
]

In [ ]:
from baybe.parameters import SubstanceParameter

encoding_choice = ["MORDRED", "RDKIT", "MORGAN_FP"]

SubstanceParameter(
    name="Inhibitor",
    data=smiles_dict,
    encoding="MORDRED",  # Can be also RDKIT or MORGAN_FP - WHICH IS BETTER?
    decorrelate=0.7,  # Change threshold to avoid overfitting?
)

These calculations will typically result in 500 to 1500 numbers per molecule. **To avoid detrimental effects on the surrogate model fit, we reduce the number of descriptors via decorrelation before using them.** For instance, the decorrelate option in the example above specifies that only descriptors with a correlation lower than 0.7 to any other descriptor will be kept. This usually reduces the number of descriptors to 10-50, depending on the specific items in data.

### Custom descriptors

In [ ]:
"""
The encoding concept introduced above is generalized by the CustomParameter. Here, the user is expected to provide their own descriptors for the encoding.

Take, for instance, a parameter that corresponds to the choice of a polymer. Polymers are not well represented by the small molecule descriptors utilized in the SubstanceParameter. 
Still, one could provide experimental measurements or common metrics used to classify polymers:
from baybe.parameters import CustomDiscreteParameter

# Create or import new dataframe containing custom descriptors

descriptors = pd.DataFrame(
    {
        "Glass_Transition_TempC": [20, -71, -39],
        "Weight_kDalton": [120, 32, 241],
    },
    index=["Polymer A", "Polymer B", "Polymer C"],  # put labels in the index
)

CustomDiscreteParameter(
    name="Polymer",
    data=descriptors,
    decorrelate=True,  # optional, uses default correlation threshold = 0.7?
)
""" 

In [ ]:
searchspace = SearchSpace.from_product(parameters)

## Objective

In [ ]:
from baybe.targets import NumericalTarget
from baybe.objective import Objective

target = NumericalTarget(
    name="Efficiency",
    mode="MAX",
)
objective = Objective(mode="SINGLE", targets=[target])

## Recommender

In [ ]:
from baybe.recommenders import RandomRecommender, SequentialGreedyRecommender
from baybe.surrogates import GaussianProcessSurrogate

available_surr_models = [
    "GaussianProcessSurrogate", 
    "BayesianLinearSurrogate",
    "MeanPredictionSurrogate",
    "NGBoostSurrogate",
    "RandomForestSurrogate"
]

available_acq_functions = [
    "qPI",  # q-Probability Of Improvement
    "qEI",  # q-Expected Improvement
    "qUCB", # q-upper confidence bound with beta of 1.0
]

# Defaults anyway
SURROGATE_MODEL = GaussianProcessSurrogate()
ACQ_FUNCTION = "qEI" # q-Expected Improvement, only q-fuctions are available for batch_size > 1

seq_greedy_recommender = SequentialGreedyRecommender(
        surrogate_model=SURROGATE_MODEL,
        acquisition_function_cls=ACQ_FUNCTION,
        hybrid_sampler="Farthest", # find more details in the documentation
        sampling_percentage=0.3, # should be relatively low
        allow_repeated_recommendations=False,
        allow_recommending_already_measured=False,
    )

# Campaign

In [ ]:
from baybe.strategies import TwoPhaseStrategy
from baybe import Campaign

strategy = TwoPhaseStrategy(
    initial_recommender = RandomRecommender(),  # Initial recommender
    # Doesn't matter since I already have training data, BUT CAN BE USED FOR BENCHMARKING
    recommender = seq_greedy_recommender,  # Bayesian model-based optimization
    switch_after=1  # Switch to the model-based recommender after 1 batches = immediately
)

campaign = Campaign(searchspace, objective, strategy)

### Get recommendations

In [ ]:
new_rec = campaign.recommend(batch_size=3) # TEST with different batch sizes for optimal performance
print("\n\nRecommended experiments: ")
print(new_rec.to_markdown())

In [ ]:
# Get and input efficiency value from Excel table, for specific SMILES component first, 
# then for the closest values of the rest of the parameters

new_rec["efficiency"] = [0.1, 0.2, 0.3]
print("\n\nRecommended experiments with measured values: ")
print(new_rec.to_markdown())

Wtf, why is recommending the same values as before here? 

In [ ]:
new_new_rec = campaign.recommend(batch_size=3) # TEST with different batch sizes for optimal performance
print("\n\nRecommended experiments: ")
print(new_new_rec.to_markdown())

### Merge all results into a dataframe

In [ ]:
results = pd.concat([new_rec, new_new_rec]) # etc.
print("\n\nAll experiments with measured values: ")
print(results.to_markdown())

# Benchmarking

# Transfer Learning

https://emdgroup.github.io/baybe/examples/Transfer_Learning/basic_transfer_learning.html

https://emdgroup.github.io/baybe/userguide/transfer_learning.html

https://emdgroup.github.io/baybe/examples/Backtesting/full_initial_data.html

https://emdgroup.github.io/baybe/examples/Backtesting/full_lookup.html

https://emdgroup.github.io/baybe/userguide/simulation.html